<a href="https://colab.research.google.com/github/rpatel71/Design-Optimization/blob/main/homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the answer of the homework-3 of Design Optimization.

Problem 1

In [11]:
# importing all the libraries required
import numpy as np
import torch as tor
from torch.autograd import Variable
import matplotlib.pyplot as plt

X1 = np.array([[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]])
X2 = np.flip(X1, axis = 1).copy()
a = np.array(([[8.0731, 1730.63, 233.426], [7.43155, 1554.679, 240.337]]))

# T = 20
p_w = 10 ** (a[0, 0] - a[0, 1] / (20 + a[0, 2]))
p_diox = 10 ** (a[1, 0] - a[1, 1] / (20 + a[1, 2]))
P = np.array([[28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]])
P = tor.tensor(P, requires_grad=False, dtype=tor.float32)

A = Variable(tor.tensor([2.0, 2.0]), requires_grad=True)

x1 = tor.tensor(X1, requires_grad=False, dtype=tor.float32)
x2 = tor.tensor(X2, requires_grad=False, dtype=tor.float32)

a = 0.001

for i in range(100):
    P_cal = x1 * tor.exp(A[0] * (A[1] * x2 / (A[0] * x1 + A[1] * x2)) ** 2) * p_w + \
          x2 * tor.exp(A[1] * (A[0] * x1 / (A[0] * x1 + A[1] * x2)) ** 2) * p_diox

    loss = (P_cal - P) ** 2
    loss = loss.sum()

    loss.backward()

    with tor.no_grad():
        A -= a * A.grad
        A.grad.zero_()

print('estimation of A12 and A21 is: ', A)
print('final loss is: ', loss.data.numpy())

estimation of A12 and A21 is:  tensor([1.9531, 1.6834], requires_grad=True)
final loss is:  0.67093503


Problem 2

In [13]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
